In [1]:
import requests
import json
from bs4 import BeautifulSoup

In [2]:
root_url = "https://www.aljazeera.com/"
# Prefix for links from Mozambique page
article_prefix = "/news/2"
# Get contents from root page
resp = requests.get(root_url + 'where/mozambique/')
soup = BeautifulSoup(resp.text, 'html.parser')

In [3]:
# Get links from the parent page
links = []
for link in soup.find_all('a'):
    if link.get('href').startswith(article_prefix):
        links.append(link.get('href'))
#Sorting list to get most most recent
links.sort(reverse=True)
links = links[:10]

In [4]:
# Fetch all raw articles
articles = []
for link in links:
    resp = requests.get(root_url + link)
    article_soup = BeautifulSoup(resp.text, 'html.parser')
    articles.append(article_soup)

In [6]:
# Preprocess so only text is in article_texts
text_tag = "wysiwyg wysiwyg--all-content css-1ck9wyi"
article_texts = []
for article_soup in articles:
    # Use tag to find text only
    text = article_soup.find("div", {"class": text_tag})
    # Get text from html tags, remove newline characters
    article_texts.append(text.get_text().replace('\n',' '))

In [ ]:
# Compute sentiment